# Notebook para calcular ceros en un archivo por columna

In [1]:
import findspark
findspark.init()
import numpy as np
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = (SparkConf()
        .setAppName("Calcular Ceros Columnas") \
        .set('spark.driver.cores', '6') \
        .set('spark.executor.cores', '6') \
        .set('spark.driver.memory', '6G') \
        .set('spark.sql.autoBroadcastJoinThreshold', '-1') \
        .set('spark.executor.memory', '6G'))
sc = SparkContext(conf=conf)

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
data = spark.read.format('libsvm')\
    .load("../data/url_svmlight/403995_x_instances_10.svm")

In [27]:
data.take(1)

[Row(label=0.0, features=SparseVector(76, {3: 0.0581, 4: 0.0897, 5: 0.1765, 10: 0.1429, 20: 0.2857, 21: 0.0238, 23: 1.0, 24: 1.0, 27: 1.0, 35: 1.0, 43: 1.0, 53: 1.0, 55: 1.0, 61: 1.0, 63: 1.0, 65: 1.0, 67: 1.0, 69: 1.0, 71: 1.0, 73: 1.0, 75: 1.0}))]

In [39]:
data2 = data.select('features').rdd.persist()

In [23]:
data2.map(lambda x: x[0][1]).take(4)

[0.0, 1.0, 1.0, 0.0]

In [29]:
col = 4
data2.map(lambda x: x[0][col]).take(1)

[0.0896552]

In [40]:
# La repartición, en este caso, tarda más milisegundos. 
data2 = data2.repartition(50)

In [41]:
print("Columna\tCeros")
# Contador para el total de ceros. 
cont_ceros = 0
# Número de columnas totales del archivo.
total_cols = 76
cont_total = 0
for i in range(total_cols):
    num = data2.filter(lambda x: x[0][i] == 0.0).count()
    # Imprimir cada cierto No. de columnas. 
    if(i < 76):
        print(i, "\t", num)
    # Aumentar el contador total de ceros que hay en todas las columnas.
    cont_total += num
print("\nEl no. de ceros totales es:", cont_total)

Columna	Ceros
0 	 10
1 	 4
2 	 10
3 	 0
4 	 0
5 	 0
6 	 10
7 	 10
8 	 9
9 	 6
10 	 0
11 	 10
12 	 10
13 	 10
14 	 10
15 	 3
16 	 1
17 	 1
18 	 1
19 	 7
20 	 0
21 	 0
22 	 6
23 	 3
24 	 5
25 	 10
26 	 9
27 	 6
28 	 10
29 	 10
30 	 9
31 	 9
32 	 6
33 	 9
34 	 10
35 	 5
36 	 7
37 	 9
38 	 10
39 	 9
40 	 7
41 	 9
42 	 10
43 	 4
44 	 7
45 	 10
46 	 10
47 	 9
48 	 10
49 	 10
50 	 10
51 	 10
52 	 10
53 	 0
54 	 10
55 	 0
56 	 10
57 	 9
58 	 10
59 	 9
60 	 10
61 	 0
62 	 10
63 	 0
64 	 10
65 	 0
66 	 10
67 	 0
68 	 10
69 	 0
70 	 10
71 	 0
72 	 10
73 	 0
74 	 7
75 	 0

El no. de ceros totales es: 495


In [ ]:
sc.stop()